In [1]:
"""
This script demonstrates a rudimentary concurrent trading system using the TWS API.
It combines multiple data streams—account updates, position updates, market scanner subscription,
and real-time market data (tickPrice)—to decide when to enter or exit trades.

The system uses threading to ensure that the API's event loop runs concurrently with our main code.
Throughout the code, extensive comments are provided to explain the logic and how each component works.
The trading strategy used here is improved compared to a basic demo:
    - It monitors account cash and will disconnect if the TotalCashBalance falls below an amount.
    - It requests position updates to track current holdings, preventing unintended short positions.
    - It uses a market scanner to subscribe to the top MOST_ACTIVE stocks on major U.S. exchanges.
    - For each of the top scanner results, it subscribes to market data.
    - In the tickPrice callback, it compares the new tick price to the previously stored price.
    - All these functions operate concurrently via the TWS API's event loop.
"""

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.tag_value import TagValue
from ibapi.order import Order
from ibapi.contract import Contract
from ibapi.scanner import ScannerSubscription

import os
import datetime
from dateutil.parser import isoparse
import time
import threading

import pandas as pd
from collections import deque

import logging
# Configure logging to output INFO-level messages on the console
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

from apscheduler.schedulers.background import BackgroundScheduler

In [2]:
class TradingSystem(EClient, EWrapper):
    """
    This class subscribes to IB account and portfolio updates.
    
    At the end of the first complete update cycle (signaled via accountDownloadEnd),
    the final StockMarketValue is saved as the shares_init_amount in self.shares_init_amount.
    
    Then, in every subsequent real-time update (in updateAccountValue) when the key
    is "StockMarketValue", the current value is compared against self.shares_init_amount. If the
    StockMarketValue drops by more than the allowed threshold, we disconnect.

    """
    
    def __init__(self, n_stocks_scanner, candle_seconds, max_perc_loss, req_snapshot):
        # Initialize the EClient with self as the wrapper.
        EClient.__init__(self, self)
        self.max_perc_loss = max_perc_loss  # For example, 0.05 means 5% loss threshold.

        # Account values.
        self.TotalCashBalance = {}          
        self.NetLiquidation = None
        self.StockMarketValue = None

        # Portfolio details.
        self.portfolio = {}
        self.curr_portfolio = None 
        self.position_ref = {}

        # For storing aggregated values per update cycle (not strictly needed for shares_init_amount check).
        self.total_owned = {}    # Overall account value (NetLiquidation)
        self.total_shares = {}   # StockMarketValue

        # Account time for each update cycle.
        self.account_time = None

        # The dedicated shares_init_amount variable (set once at the end of the first update cycle).
        self.shares_init_amount = None

        self.n_stocks_scanner = n_stocks_scanner
        self.candle_seconds = candle_seconds
        self.req_snapshot = req_snapshot

        # Scanner contracts retrieved
        self.ScannerResults = {}
        self.daily_history = []
        # For real-time processing: fixed-size deque (depending by the number of stocks scanned, the length of the candlesticks in second, and a multiplier eg 300)
        # maxlen set in order to have sufficient data to generate the trading indicators, before the deque gets reset
        self.ticks_history = deque(maxlen = self.n_stocks_scanner * self.candle_seconds * 300)

        self.next_id = 0

        


    def nextValidId(self, orderId: int):
        pass


    
    def nextId(self) -> int:
        self.next_id += 1
        return self.next_id


        
    def error(self, reqId: int, errorTime: int, errorCode: int, 
              errorString: str, advancedOrderReject=""):
        pass
        # print(f"Error - reqId:{reqId}, errorTime:{errorTime}, errorCode:{errorCode}, "
        #       f"errorString:{errorString}, OrderReject:{advancedOrderReject}")

    
    
    def updateAccountValue(self, key: str, val: str, currency: str, accountName: str):
        """
        Called repeatedly during the account update cycle.
        This callback simply stores the incoming values.
        Also, for "StockMarketValue" updates that come in after the shares_init_amount has been set,
        a real-time safety check is performed.
        """
        if key == "TotalCashBalance":
            self.TotalCashBalance[currency] = float(val)
        elif key == "NetLiquidation":
            self.NetLiquidation = float(val)
        elif key == "StockMarketValue":
            self.StockMarketValue = float(val)
            
        if (self.shares_init_amount is not None  # wait for the initial amount to be processed in the 'accountDownloadEnd' method 
        and self.portfolio != self.curr_portfolio): # to add the new portfolio value only if it has been updated
            self.total_owned[self.account_time] = self.NetLiquidation
            shares_curr_amount = round(sum(self.portfolio.values()),2)
            self.total_shares[self.account_time] = shares_curr_amount
            print('[EXEC "updateAccountValue"] TotalCashBalance:', self.TotalCashBalance)
            print('[EXEC "updateAccountValue"] Current Portfolio:', self.portfolio)
            print('[EXEC "updateAccountValue"] Total EUR owned amounts:', self.total_owned)
            print('[EXEC "updateAccountValue"] Total USD shares amounts:', self.total_shares)
            self.curr_portfolio = self.portfolio.copy()
    
            #safety check (disconnect if shares amount drops below a threshold)
            if shares_curr_amount < self.shares_init_amount * (1 - self.max_perc_loss):
                print(f"shares_curr_amount {shares_curr_amount:.2f} "
                      f"is lower than the shares_init_amount {self.shares_init_amount:.2f} "
                      f"(a drop of more than {self.max_perc_loss:.2f}%).")
                print("Disconnecting for safety!")
                self.disconnect()

    
                
    def updatePortfolio(self, contract: Contract, position: float, marketPrice: float,
                        marketValue: float, averageCost: float, unrealizedPNL: float,
                        realizedPNL: float, accountName: str):
        """
        Called for each portfolio update.
        We store the market value for each stock keyed by the contract symbol.
        """
        self.portfolio[contract.symbol] = marketValue

    
    
    def updateAccountTime(self, timeStamp: str):
        """
        Saves the account time for the current update cycle.
        """
        self.account_time = timeStamp

    
    
    def accountDownloadEnd(self, accountName: str):
        """
        Called when the entire batch of account (and portfolio) updates is finished.
        We perform our final processing here.
        """
        # Set the shares_init_amount (if set here, and not directly in the 'updateAccountValue' method, it stores the correct result)
        if self.shares_init_amount is None:
            self.shares_init_amount = self.StockMarketValue
            print('[EXEC "accountDownloadEnd"] shares_init_amount:', self.shares_init_amount)



    def position(self, account, contract: Contract, position, avgCost):
        """
        Callback: Receives updates for positions held.
        We record the current position for each symbol in our position_ref dictionary,
        ensuring that trading logic can check current holdings to avoid short positions.
        """
        self.position_ref[contract.symbol] = float(position)
        # print(f'[EXEC "position"] {contract.symbol}: Position = {position}, AvgCost = {avgCost}')
        next_id = self.nextId()
        self.ScannerResults[next_id] = {
                                        "ConId": contract.conId,
                                        "Symbol": contract.symbol,
                                        "SecType": contract.secType,
                                        "Exchange": contract.exchange,
                                        "Currency": contract.currency,
                                        # add any other properties you need...
                                        }        
        self.reqMktData(reqId = next_id, # to provide a different reqId for each contract
                       contract = contract, 
                       genericTickList = "", # A coma delimited list of generic tick types (?)
                       snapshot = self.req_snapshot, 
                       regulatorySnapshot = False, 
                       mktDataOptions = []) 



    def positionEnd(self):
        """This is called once all position data for a given request are
        received and functions as an end marker for the position() data."""
        pass

    

    def scannerParameters(self, xml: str):
        """Provides the xml-formatted parameters available to create a marketscanner """
        with open("scanner_parameters.xml", "w", encoding="utf-8") as file:
            file.write(xml)

        
    
    def scannerData(self, reqId, rank, contractDetails, distance, benchmark, projection, legsStr):
        """
        Callback: Receives results from a market scanner subscription.
        We limit our processing to the top 'n_stocks_scanner' results.
        We add the scanned contracts to the position_ref dictionaty with an initial value of 0.
        Later, when we receive an update about the actual holdings through the 'position' callback, the same dictionary entry is updated with the real position value. 
        This way, if we’re already holding some shares of that instrument, the actual position update will overwrite the 0 that the scanner set.
        """
        self.contractDetails = contractDetails
        if rank < self.n_stocks_scanner:
            next_id = self.nextId()
            self.ScannerResults[next_id] = {
                                            "ConId": contractDetails.contract.conId,
                                            "Symbol": contractDetails.contract.symbol,
                                            "SecType": contractDetails.contract.secType,
                                            "Exchange": contractDetails.contract.exchange,
                                            "Currency": contractDetails.contract.currency,
                                            # add any other properties you need...
                                            }
            self.position_ref[self.contractDetails.contract.symbol] = 0 # Initialize our position for this contract to zero (avoid short positions)       
            
            # Call this function to request market data. The market data will be returned by the Callbacks --> 'tickPrice' and 'tickSize'
            self.reqMktData(reqId = next_id, # to provide a different reqId for each contract
                           contract = self.contractDetails.contract, 
                           genericTickList = "", # A coma delimited list of generic tick types (?)
                           snapshot = self.req_snapshot, 
                           regulatorySnapshot = False, 
                           mktDataOptions = []) 
            print(f'[EXEC "scannerData"] Rank {rank}: {self.contractDetails.contract.symbol} on {self.contractDetails.contract.exchange}')


    
    def scannerDataEnd(self, reqId):
        """
        Callback: Indicates that the market scanner subscription has finished sending data.
        Here, we cancel the scanner subscription to avoid duplicate data, but we do not disconnect.
        """
        self.cancelScannerSubscription(reqId)

    

    def tickString(self, reqId, tickType, value):
        pass
        # if reqId == self.next_id: # just printing the ScannerResults dictionary at the last reqId callback
        #     # we print ScannerResults here, even if its update is linked also to the callbacks: tickSize and tickPrice
        #     print('[EXEC "tickString"] ScannerResults:', self.ScannerResults) 


    
    def tickSize(self, reqId, tickType, size):
        """
        Market data tick size callback. Handles all size-related ticks.
        It adds available size data to the ScannerResults.
        """
        if tickType == 8:  # Trading Volume
            self.ScannerResults[reqId]["Volume"] = int(size)
            self.ScannerResults[reqId]["TS_Volume"] = datetime.datetime.now().isoformat()
        
        # Record the update (this unified log call will filter out duplicates).
        self.record_tick_update(reqId)


        
    def tickPrice(self, reqId, tickType, price, attrib):
        """
        Market data tick price callback. Handles price-related ticks.
        Updates ScannerResults with the following fields:
          - Bid Price (tick type 1)
          - Ask Price (tick type 2)
          - Last Price (tick type 4)
        """
        if tickType == 1:
            self.ScannerResults[reqId]["Bid_price"] = price
            self.ScannerResults[reqId]["TS_Bid_price"] = datetime.datetime.now().isoformat()
        elif tickType == 2:
            self.ScannerResults[reqId]["Ask_price"] = price
            self.ScannerResults[reqId]["TS_Ask_price"] = datetime.datetime.now().isoformat()
        elif tickType == 4:
            self.ScannerResults[reqId]["Last_price"] = price
            self.ScannerResults[reqId]["TS_Last_price"] = datetime.datetime.now().isoformat()
    
        # Record the update.
        self.record_tick_update(reqId)
        

    
    def record_tick_update(self, reqId):
        # Create a new snapshot from the current market data.
        new_snapshot = self.ScannerResults[reqId].copy()
        
        # Gather all timestamp values (from keys starting with "TS_")
        ts_values = []
        for key, value in new_snapshot.items():
            if key.startswith("TS_") and value is not None: 
                dt = datetime.datetime.fromisoformat(value) if isinstance(value, str) else value
                ts_values.append(dt)
        
        # Compute the maximum timestamp (i.e., the latest update time) and add it as a new field.
        latest_ts = max(ts_values)
        # Store the consolidated timestamp in ISO format.
        new_snapshot["TimeStamp"] = latest_ts.isoformat()
    
        # Remove all individual timestamp keys from the snapshot.
        for key in list(new_snapshot.keys()):
            if key.startswith("TS_"):
                del new_snapshot[key]
        
        # Define the keys to check for duplication.
        # Exclude the 'TimeStamp' field since it is updated every time.
        keys_to_check = [k for k in new_snapshot if k != "TimeStamp"]
        
        # If ticks_history is not empty, compare with the last snapshot.
        if self.ticks_history:
            last_snapshot = self.ticks_history[-1]
            # Compare each significant key between the new snapshot and the last snapshot.
            if all(new_snapshot.get(k) == last_snapshot.get(k) for k in keys_to_check):
                # If all values (ignoring TimeStamp) are the same, consider it a duplicate.
                return  # Do not log this update.
        
        # Otherwise, log the new snapshot.
        self.ticks_history.append(new_snapshot)
        self.daily_history.append(new_snapshot)



    def tickSnapshotEnd(self, reqId):
        """
        When requesting market data snapshots, this market will indicate the
        snapshot reception is finished.
        """
        if reqId == self.next_id: # just printing the ScannerResults dictionary at the last tickSnapshotEnd callback
            print('[EXEC "tickSnapshotEnd"] ScannerResults:', self.ScannerResults)

        

    def archive_daily_data(self, folder):
        """
        Archives the daily_history list to a CSV file. The file is named using the previous day's date.
        """
        if not os.path.exists(folder):
            os.makedirs(folder)
        
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  
        filename = os.path.join(folder, f"daily_trading_data_{timestamp}.csv")
        
        if self.daily_history:
            df_daily = pd.DataFrame.from_records(self.daily_history).dropna()
            df_daily["TimeStamp"] = df_daily["TimeStamp"].apply(isoparse)   
            df_daily.to_csv(filename, index=False)
            print(f"Archived daily data to {filename}.")
            self.daily_history = [] # reset the list for the next day data
        else:
            print("No data to archive for the previous day.")


    
    def start_scheduler(self, folder, hour=2, minute=0):
        """
        Start the background scheduler to run archive_daily_data daily.
        """
        self.scheduler = BackgroundScheduler()
        # Schedule the archive job daily. archive_daily_data receives 'folder' as an argument.
        self.scheduler.add_job(self.archive_daily_data, 'cron', hour=hour, minute=minute, args=[folder])
        self.scheduler.start()
        print("Scheduler started: archive_daily_data will run daily.")




    # # Create a new market order object
    # order = Order()
    # order.tif = "DAY"           # Time in Force: Order is valid for the day
    # order.orderType = "MKT"     # Market Order
    # # Strategy Improvement:
    # # If we are not holding any position for this symbol, set an initial order quantity.
    # # If we already have a position, we may choose to add or reduce the holding.
    # initial_qty = 1  # Default order quantity

    # # Improved Logic:
    # # Check if the new price is significantly higher than the previous price (more than 5% gain).
    # if self.last_price > previous_price * 1.05:
    #     # Only consider buying if we are not already in a position or choose to scale in.
    #     if position_ref.get(contract.symbol, 0) == 0:
    #         order.action = "BUY"
    #         order.totalQuantity = initial_qty
    #         print(f"[Signal BUY] {contract.symbol}: Price increased from {previous_price:.2f} to {self.last_price:.2f}. No current position; buying {order.totalQuantity} shares.")
    #         self.placeOrder(self.nextId(), contract, order)
    #     else:
    #         # Optionally, add to an existing position if desired. Here, we add a smaller quantity.
    #         order.action = "BUY"
    #         order.totalQuantity = 3
    #         print(f"[Signal BUY Add] {contract.symbol}: Price increased from {previous_price:.2f} to {self.last_price:.2f}. Already holding {position_ref[contract.symbol]} shares; adding {order.totalQuantity} shares.")
    #         self.placeOrder(self.nextId(), contract, order)
    
    # # Check if the new price is significantly lower than the previous price (more than 6% drop)
    # elif self.last_price < previous_price * 0.94:
    #     # Only sell if we have a current long position; avoid going short.
    #     if position_ref.get(contract.symbol, 0) >= initial_qty:
    #         order.action = "SELL"
    #         # Sell a fixed quantity (could be full position if desired)
    #         order.totalQuantity = initial_qty
    #         print(f"[Signal SELL] {contract.symbol}: Price dropped from {previous_price:.2f} to {self.last_price:.2f}. Holding {position_ref[contract.symbol]} shares; selling {order.totalQuantity} shares.")
    #         self.placeOrder(self.nextId(), contract, order)
    #     else:
    #         print(f"[No SELL] {contract.symbol}: Price dropped from {previous_price:.2f} to {self.last_price:.2f} but insufficient position to sell.")
    
    # # Update the stored LAST price for future comparisons.
    # self.ScannerResults[reqId]["LAST"] = self.last_price



    # def openOrder(self, orderId, contract: Contract, order, orderState):
    #     """
    #     Callback: Informs us about the status of open orders.
    #     Here, we log any rejected orders with a timestamp for review.
    #     """
    #     if orderState.status == "Rejected":
    #         print(f"[Order Rejected] {datetime.datetime.now()} ID:{orderId} || {order.action} {order.totalQuantity} {contract.symbol}")

    # def execDetails(self, reqId, contract: Contract, execution):
    #     """
    #     Callback: Returns execution details upon order fills.
    #     This example prints a simple summary of the trade execution.
    #     """
    #     print(f"[Execution] Order ID:{execution.orderId} || {execution.side} {execution.shares} {contract.symbol} @ {execution.time}")




In [3]:
port = 7496  # Typical port for connecting to TWS (7496 for IB Gateway live trading)
clientId = 101

n_stocks_scanner = 10 # top n stocks to retrieve by the scanner
candle_seconds = 60 # number of seconds for the trading candlesticks aggregation
max_perc_loss = 5 # max percentage loss allowed (automatically disconnected if exceeded)
sched_hour = 23
sched_min = 59
req_snapshot = False

# Create an instance of the TradingSystem and connect to TWS.
trading_system = TradingSystem(n_stocks_scanner, candle_seconds, max_perc_loss, req_snapshot)
trading_system .connect("localhost", port, clientId) # "localhost" == "127.0.0.1"

# Start the API processing loop in a separate thread so that it does not block the main thread.
#by setting daemon=True, we ensure that the event loop thread running trading_system .run() will not hold up the termination of the overall program. 
# because in TWS API integration, we want the main thread to handle shutdown logic gracefully without waiting for every background thread to finish.
threading.Thread(target=trading_system.run, daemon=True).start()

# Start the scheduler to trigger archive_daily_data.
trading_system.start_scheduler(folder=os.path.abspath("daily_archives"), hour=sched_hour, minute=sched_min)


2025-05-21 16:18:35,846 [INFO] sent startApi
2025-05-21 16:18:35,850 [INFO] REQUEST startApi {}
2025-05-21 16:18:35,851 [INFO] SENDING startApi b'\x00\x00\x00\x0b\x00\x00\x00G2\x00101\x00\x00'
2025-05-21 16:18:35,853 [INFO] ANSWER connectAck {}
2025-05-21 16:18:35,855 [INFO] ANSWER openOrderEnd {}
2025-05-21 16:18:35,876 [INFO] Adding job tentatively -- it will be properly scheduled when the scheduler starts
2025-05-21 16:18:35,878 [INFO] Added job "TradingSystem.archive_daily_data" to job store "default"
2025-05-21 16:18:35,879 [INFO] Scheduler started


Scheduler started: archive_daily_data will run daily.


In [4]:
account_check = False
parameters_check = False
scanner_check = True
portfolio_check = False

# Set up a market scanner subscription to identify trending opportunities.
sub = ScannerSubscription()
sub.instrument = "STK"             # We are interested in stocks.
sub.locationCode = "STK.US.MAJOR"    # Focus on major US exchanges (NYSE, NASDAQ, AMEX, etc.).
sub.scanCode = "MOST_ACTIVE"         # Use the "MOST_ACTIVE" scan to get highly traded stocks.

# Define filter options to narrow down the scanner results:
filter_options = [
    TagValue("avgVolumeAbove", "1000000"),  # Stocks with an average trading volume above 1,000,000
    TagValue("priceAbove", "30"),            
    TagValue("priceBelow", "900")          
]

scan_options = [] # For internal use only, so we leave the list empty.

if account_check:
    '''
    Request account updates to maintain a continuous stream of account cash data.
    We use this to check if the TotalCashBalance in our "BASE" currency (e.g., USD) remains sufficient.
    Callbacks --> 'updateAccountValue', 'updatePortfolio' and 'updateAccountTime'
    Set the parameter 'subscribe' to true to start the subscription and to false to stop it.
     
    The unrealized P&L information shown in the TWS Account Window which is sent to updatePortfolioValue will update either 
    (1) when a trade for that particular instrument occurs or 
    (2) every 3 minutes. 
    The realized P&L data in the TWS Account Window is reset to 0 once per day.
    '''
    
    trading_system.reqAccountUpdates(subscribe=True, # "True": the client will start receiving account and Portfolio updates, "False": it stops doing it
                          acctCode="") # The account code for which to receive account and portfolio updates

if parameters_check:
    """
    Requests an XML string that describes all possible scanner queries.
    A string containing all available XML-formatted parameters will then be returned via EWrapper.scannerParameters,
    which can be used as parameters in the "reqScannerSubscription"
    Callback --> scannerParameters (saves the file "scanner_parameters.xml")
    """
    trading_system.reqScannerParameters()


if scanner_check:
    '''
    Request the market scanner subscription using a unique request ID.
    Note: The program will continue to run with ongoing account updates, position updates, market scanner data, and tick price processing concurrently,
    until the callback cancelScannerSubscription(self, reqId: int) is invoked
    Callbacks --> 'scannerData', 'scannerDataEnd'
    '''
    trading_system.reqScannerSubscription(reqId=0, # any unique id
                               subscription=sub,
                               scannerSubscriptionOptions=scan_options,
                               scannerSubscriptionFilterOptions=filter_options)

if portfolio_check:
    '''
    Request position updates so that position_ref is kept up to date with our current portfolio holdings.
    Requests real-time position data for all accounts.
    Callbacks --> 'position', 'positionEnd'
    '''
    trading_system.reqPositions()


time.sleep(n_stocks_scanner) # if necessary

2025-05-21 16:18:35,930 [INFO] ANSWER managedAccounts {'accountsList': 'U18112846'}
2025-05-21 16:18:35,935 [INFO] REQUEST reqScannerSubscription {'reqId': 0, 'subscription': 2500363355376: Instrument: STK, LocationCode: STK.US.MAJOR, ScanCode: MOST_ACTIVE, 'scannerSubscriptionOptions': [], 'scannerSubscriptionFilterOptions': [2500363355328: avgVolumeAbove=1000000;, 2500363359696: priceAbove=30;, 2500363359600: priceBelow=900;]}
2025-05-21 16:18:35,939 [INFO] SENDING reqScannerSubscription b'\x00\x00\x00n\x00\x00\x00\x160\x00-1\x00STK\x00STK.US.MAJOR\x00MOST_ACTIVE\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000\x00\x00\x00\x00avgVolumeAbove=1000000;priceAbove=30;priceBelow=900;\x00\x00'
2025-05-21 16:18:36,354 [INFO] REQUEST reqMktData {'reqId': 1, 'contract': 2500363405968: ConId: 4815747, Symbol: NVDA, SecType: STK, LastTradeDateOrContractMonth: , Strike: 0, Right: , Multiplier: , Exchange: SMART, PrimaryExchange: , Currency: USD, LocalSymbol: NVDA, TradingClass: NMS, Inclu

[EXEC "scannerData"] Rank 0: NVDA on SMART
[EXEC "scannerData"] Rank 1: TSLA on SMART
[EXEC "scannerData"] Rank 2: TQQQ on SMART
[EXEC "scannerData"] Rank 3: UNH on SMART
[EXEC "scannerData"] Rank 4: PLTR on SMART
[EXEC "scannerData"] Rank 5: GOOGL on SMART
[EXEC "scannerData"] Rank 6: SMCI on SMART
[EXEC "scannerData"] Rank 7: IBIT on SMART
[EXEC "scannerData"] Rank 8: CRWV on SMART
[EXEC "scannerData"] Rank 9: SPY on SMART


2025-05-21 16:18:36,679 [INFO] ANSWER marketDataType {'reqId': 4, 'marketDataType': 1}
2025-05-21 16:18:36,679 [INFO] ANSWER tickReqParams {'tickerId': 4, 'minTick': 0.01, 'bboExchange': 'a60001', 'snapshotPermissions': 3}
2025-05-21 16:18:36,686 [INFO] ANSWER marketDataType {'reqId': 5, 'marketDataType': 1}
2025-05-21 16:18:36,688 [INFO] ANSWER tickReqParams {'tickerId': 5, 'minTick': 0.01, 'bboExchange': '9c0001', 'snapshotPermissions': 3}
2025-05-21 16:18:36,688 [INFO] ANSWER marketDataType {'reqId': 6, 'marketDataType': 1}
2025-05-21 16:18:36,688 [INFO] ANSWER tickReqParams {'tickerId': 6, 'minTick': 0.01, 'bboExchange': '9c0001', 'snapshotPermissions': 3}
2025-05-21 16:18:36,688 [INFO] ANSWER marketDataType {'reqId': 7, 'marketDataType': 1}
2025-05-21 16:18:36,688 [INFO] ANSWER tickReqParams {'tickerId': 7, 'minTick': 0.01, 'bboExchange': '9c0001', 'snapshotPermissions': 3}
2025-05-21 16:18:36,695 [INFO] ANSWER marketDataType {'reqId': 8, 'marketDataType': 1}
2025-05-21 16:18:36,6

In [5]:
# --- Create the DataFrame from the ticks_history deque ---
df_ticks = pd.DataFrame.from_records(list(trading_system.ticks_history)).dropna()

# Convert the TimeStamp column from string to datetime.
df_ticks["TimeStamp"] = df_ticks["TimeStamp"].apply(isoparse)

# Optionally, print out some DataFrame info.
print(df_ticks.info(), "\n")
print("Number of unique rows in df_ticks:", df_ticks.drop_duplicates().shape[0], "\n")
print(df_ticks.nunique(), "\n")

# Set the TimeStamp as the index (required for resampling).
df_ticks.set_index("TimeStamp", inplace=True)

# Define the frequency string for resampling (e.g. "5s" for 5-second candles).
freq = f"{trading_system.candle_seconds}s"  # trading_system.candle_seconds is the candle interval in seconds.

# --- Group by Symbol and Resample ---
ohlc_by_symbol = {}

# A helper function to aggregate volume.
# being "Volume" a cumulative (running) total for the day, we cannot simply sum the values over the candle period
# we need to compute the difference between the last and first volume values in each resampled interval
def volume_diff(x):
    # If there's only one record in the bucket, the traded volume is 0.
    if len(x) < 2:
        return 0
    return x.iloc[-1] - x.iloc[0]

for symbol, group in df_ticks.groupby("Symbol"):
    # Resample the 'Last_price' per symbol to compute OHLC candles for the defined interval.
    ohlc = group["Last_price"].resample(freq).ohlc()
    
    # Use the custom aggregation for cumulative volume:
    # Calculate the difference between the last and first tick of each candle.
    if "Volume" in group.columns:
        ohlc["volume"] = group["Volume"].resample(freq).agg(volume_diff)
    
    # Optionally, add the Symbol as a column for easier identification.
    ohlc["symbol"] = symbol
    
    # Store the resulting DataFrame in the dictionary.
    ohlc_by_symbol[symbol] = ohlc

# --- Example: Display the first few rows of the OHLC data for each Symbol ---
for symbol, ohlc_df in ohlc_by_symbol.items():
    print(ohlc_df.shape)
    print(ohlc_df.head())
    print(ohlc_df.tail(),'\n')

# df_ticks

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 3 to 640
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ConId       614 non-null    int64         
 1   Symbol      614 non-null    object        
 2   SecType     614 non-null    object        
 3   Exchange    614 non-null    object        
 4   Currency    614 non-null    object        
 5   Bid_price   614 non-null    float64       
 6   TimeStamp   614 non-null    datetime64[ns]
 7   Ask_price   614 non-null    float64       
 8   Last_price  614 non-null    float64       
 9   Volume      614 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 52.8+ KB
None 

Number of unique rows in df_ticks: 524 

ConId           9
Symbol          9
SecType         1
Exchange        1
Currency        1
Bid_price      53
TimeStamp     105
Ask_price      51
Last_price     78
Volume        208
dtype: int64 


In [6]:

# # Calculate the date string for previous day.
# previous_day = datetime.datetime.now() - datetime.timedelta(days=1)
# date_str = previous_day.strftime("%Y-%m-%d")

# # Define the directory and ensure it exists.
# directory = os.path.abspath("daily_archives")
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # Use an f-string and os.path.join to build the filename.
# filename = os.path.join(directory, f"daily_trading_data_{date_str}.csv")

# # Create the DataFrame and process the timestamp.
# df_daily = pd.DataFrame.from_records(trading_system.daily_history).dropna()
# df_daily["TimeStamp"] = df_daily["TimeStamp"].apply(isoparse)

# # Save to CSV.
# df_daily.to_csv(filename, index=False)
